In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import cv2
from tqdm import tqdm
import matplotlib.pyplot as plt

In [3]:
train = pd.read_csv('dataset/train.csv')
train.head()

,Image,target
0,96.jpg,manipuri
1,163.jpg,bharatanatyam
2,450.jpg,odissi
3,219.jpg,kathakali
4,455.jpg,odissi


In [4]:
train['target'].value_counts()

mohiniyattam     50
odissi           49
kathakali        47
bharatanatyam    47
kuchipudi        46
sattriya         45
kathak           44
manipuri         36
Name: target, dtype: int64

In [5]:
test = pd.read_csv('dataset/test.csv')

In [6]:
Class_map={'manipuri':0,'kathak':1,'sattriya':2,'kuchipudi':3,'kathakali':4,'bharatanatyam':5,'odissi':6,'mohiniyattam':7}
inverse_map={0:'manipuri',1:'kathak',2:'sattriya',3:'kuchipudi',4:'kathakali',5:'bharatanatyam',6:'odissi',7:'mohiniyattam'}
train['target']=train['target'].map(Class_map)

In [10]:
train_img=[]
train_label=[]
j=0
path='dataset/train'
for i in tqdm(train['Image']):
    final_path=os.path.join(path,i)
    img=cv2.imread(final_path)
    img=cv2.resize(img,(224,224))
    img=img.astype('float32')
    train_img.append(img)
    train_label.append(train['target'][j])
    j=j+1

100%|███████████████████████████████████████████████████████████████████████████████| 364/364 [00:02<00:00, 130.38it/s]


In [11]:
test_img=[]
path='dataset/test'
for i in tqdm(test['Image']):
    final_path=os.path.join(path,i)
    img=cv2.imread(final_path)
    img=cv2.resize(img,(224,224))
    img=img.astype('float32')
    test_img.append(img)

100%|████████████████████████████████████████████████████████████████████████████████| 156/156 [00:03<00:00, 42.37it/s]


In [16]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagenerator = ImageDataGenerator(
        featurewise_center=False,  
        samplewise_center=False,  
        featurewise_std_normalization=False,  
        samplewise_std_normalization=False,  
        rotation_range=10,  
        zoom_range = 0.10,  
        width_shift_range=0.10,  
        height_shift_range=0.10,  
        horizontal_flip=True,  
        vertical_flip=False) 


datagenerator.fit(train_img)

In [13]:
train_img=np.array(train_img)
test_img=np.array(test_img)
train_label=np.array(train_label)
print(train_img.shape)
print(test_img.shape)
print(train_label.shape)

(364, 224, 224, 3)
(156, 224, 224, 3)
(364,)


In [14]:
from tensorflow.keras.applications.vgg16 import VGG16

from tensorflow.keras.layers import Flatten,Dense,Dropout,BatchNormalization
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.utils import to_categorical
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization

from tensorflow.keras.callbacks import ReduceLROnPlateau

Using TensorFlow backend.


In [18]:
from keras.applications import VGG19
from keras.applications.vgg19 import preprocess_input
from keras.layers import *
from keras.models import Sequential
vgg19 = VGG19(weights='imagenet', include_top=False, input_shape = (224, 224, 3),pooling='avg')
vgg19.trainable = False

model = Sequential([
  vgg19, 
  Dense(1024, activation='relu'),
  Dropout(0.4),
  Dense(256, activation='relu'),
  Dense(8, activation='softmax'),
])

In [19]:
from keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop

vgg19.trainable=False

reduce_learning_rate = ReduceLROnPlateau(monitor='loss',
                                         factor=0.1,
                                         patience=2,
                                         cooldown=2,
                                         min_lr=0.00001,
                                         verbose=1)

callbacks = [reduce_learning_rate]
    


model.compile( optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit_generator(datagenerator.flow(train_img, to_categorical(train_label,8), batch_size=64),
                    epochs=5,callbacks=callbacks)



Epoch 1/5
6/6 [==============================] - 578s 96s/step - loss: 6.0425 - accuracy: 0.2582
Epoch 2/5
6/6 [==============================] - 606s 101s/step - loss: 2.2257 - accuracy: 0.4258
Epoch 3/5
6/6 [==============================] - 578s 96s/step - loss: 1.4592 - accuracy: 0.5440
Epoch 4/5
6/6 [==============================] - 592s 99s/step - loss: 1.2125 - accuracy: 0.6044
Epoch 5/5
6/6 [==============================] - 656s 109s/step - loss: 0.8419 - accuracy: 0.7033


In [20]:
labels = model.predict(test_img)
label = [np.argmax(i) for i in labels]
class_label = [inverse_map[x] for x in label]
submission = pd.DataFrame({ 'Image': test.Image, 'target': class_label })
submission.to_csv('result.csv', index=False)